# 1. 필요한 라이브러리 import 및 데이터 확인

In [2]:
import glob
import os, re
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split

txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:10])

데이터 크기: 187088
Examples:
 [' There must be some kind of way outta here', 'Said the joker to the thief', "There's too much confusion", "I can't get no relief Business men, they drink my wine", 'Plowman dig my earth', 'None were level on the mind', 'Nobody up at his word', 'Hey, hey No reason to get excited', 'The thief he kindly spoke', 'There are many here among us']


# 2. 문장 정제

In [3]:
# 입력된 문장을
#     1. 소문자로 바꾸고, 양쪽 공백을 지웁니다
#     2. 특수문자 양쪽에 공백을 넣고
#     3. 여러개의 공백은 하나의 공백으로 바꿉니다
#     4. a-zA-Z?.!,¿가 아닌 모든 문자를 하나의 공백으로 바꿉니다
#     5. 다시 양쪽 공백을 지웁니다
#     6. 문장 시작에는 <start>, 끝에는 <end>를 추가합니다
# 이 순서로 처리해주면 문제가 되는 상황을 방지할 수 있겠네요!
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() # 1
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 2
    sentence = re.sub(r'[" "]+', " ", sentence) # 3
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # 4
    sentence = sentence.strip() # 5
    if len(sentence) == 0 :
        return 0
    elif sentence.count(" ") > 12:
        return 0
    else:
        sentence = '<start> ' + sentence + ' <end>' # 6
        return sentence

# 이 문장이 어떻게 필터링되는지 확인해 보세요.
print(preprocess_sentence("This @_is ;;;sample        sentence."))

<start> this is sample sentence . <end>


## 함수를 만들때, 팀원의 도움을 받았습니다. 
### 처음에 제가 만들때는 함수를 만들고 뒤에가서 걸러 내려고 했는데, 함수를 짜면서 원하지 않는 문장을 바로 0으로 리턴해 그 다음에 필요한 문장만 뽑는다면 메모리와 처리 속도를 올릴 수 있어 필요한 문장을 뽑는 함수를 만들었습니다.

# 3. 정제된 문장을 모으고 일부 출력해보겠습니다.

In [4]:
# 여기에 정제된 문장을 모을겁니다
corpus = []

for sentence in raw_corpus:
    preprocessed_sentence = preprocess_sentence(sentence)
    if preprocessed_sentence == 0:
        pass
    else:
        corpus.append(preprocessed_sentence)
        
# 정제된 결과를 10개만 확인해보죠
corpus[:10]

['<start> there must be some kind of way outta here <end>',
 '<start> said the joker to the thief <end>',
 '<start> there s too much confusion <end>',
 '<start> i can t get no relief business men , they drink my wine <end>',
 '<start> plowman dig my earth <end>',
 '<start> none were level on the mind <end>',
 '<start> nobody up at his word <end>',
 '<start> hey , hey no reason to get excited <end>',
 '<start> the thief he kindly spoke <end>',
 '<start> there are many here among us <end>']

# 4. 토크나이저 생성

In [5]:
# 토큰화 할 때 텐서플로우의 Tokenizer와 pad_sequences를 사용합니다
# 더 잘 알기 위해 아래 문서들을 참고하면 좋습니다
# https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/text/Tokenizer
# https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/sequence/pad_sequences
def tokenize(corpus):
    # 12000단어를 기억할 수 있는 tokenizer를 만들겁니다
    # 우리는 이미 문장을 정제했으니 filters가 필요없어요
    # 12000단어에 포함되지 못한 단어는 '<unk>'로 바꿀거에요
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000, 
        filters=' ',
        oov_token="<unk>"
    )
    # corpus를 이용해 tokenizer 내부의 단어장을 완성합니다
    tokenizer.fit_on_texts(corpus)
    # 준비한 tokenizer를 이용해 corpus를 Tensor로 변환합니다
    tensor = tokenizer.texts_to_sequences(corpus)   
    # 입력 데이터의 시퀀스 길이를 일정하게 맞춰줍니다
    # 만약 시퀀스가 짧다면 문장 뒤에 패딩을 붙여 길이를 맞춰줍니다.
    # 문장 앞에 패딩을 붙여 길이를 맞추고 싶다면 padding='pre'를 사용합니다
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post', maxlen=15)  
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[  2  62 271 ...   0   0   0]
 [  2 117   6 ...   0   0   0]
 [  2  62  17 ...   0   0   0]
 ...
 [  2  75  45 ...   3   0   0]
 [  2  49   5 ...   0   0   0]
 [  2  13 635 ...   0   0   0]] <keras_preprocessing.text.Tokenizer object at 0x7fdf0a7e8990>


## 여러 옵션들을 텐서플로우 튜토리얼에서 확인했습니다.

![image](https://user-images.githubusercontent.com/84653776/135948844-6df468fc-a50c-427a-88da-20466a1f0b16.png)

## 문장이 잘 바뀌었나 확인해 보겠습니다.

In [6]:
print(tensor[:3, :10])

[[   2   62  271   27   94  546   20   86  743   90]
 [   2  117    6 6269   10    6 2310    3    0    0]
 [   2   62   17  102  184 2718    3    0    0    0]]


## 생성된 단어장을 확인해 보겠습니다.

In [7]:
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 10: break

1 : <unk>
2 : <start>
3 : <end>
4 : i
5 : ,
6 : the
7 : you
8 : and
9 : a
10 : to


### 단어장을 생성해 보았는데 여러번 돌려도 같은 순서같고, 12000단어의 중요도를 어떻게 판단하는지 궁금했는데 수요일 TF 마스터 수업에서 조원의 도움으로 알게되었다.

In [8]:
config = tokenizer.get_config()
### print(config)  너무 길어서 짤라서 올리겠습니다.

{'num_words': 12000, 'filters': ' ', 'lower': True, 'split': ' ', 'char_level': False, 'oov_token': '<unk>', 'document_count': 156174, 'word_counts': '{"<'star't>": 156174, "there": 3040, "must": 509, "be": 6646, "some": 1847, "kind": 224, "of": 8964, "way": 2081, "outta": 145, "here": 1914, "<'end'>": 156174, "said": 1485, "the": 39314, "joker": 7, "to": 20368, "thief": 31, "s": 11126, "too": 1681, "much": 818, "confusion": 25, "i": 45489, "can": 5616, "t": 11207, "get": 4928, "no": 5083, "relief": 8, "business": 136, "men": 184, ",": 43415, "they": 5200, "drink": 264, "my": 15171, "wine": 106, "plowman": 1, "dig": 88, "earth": 180, "none": 102, "were": 912, "level": 45, "on": 9773, "mind": 871, "nobody": 588, "up": 6341, "at": 2758, "his": 1750, "word": 257, "hey": 1145, "reason": 210, "excited": 28, "he": 3802, "kindly": 12, "spoke": 47, "are": 2515, "many": 433, "among": 37, "us": 1353, "who": 1867, "feel": 1619, "that": 11539, "life": 1595, "is": 7287, "but": 5695, "a": 20617, "joke": 42, "and": 23103, "this": 5083, "not": 2793, "our": 1113, "fate": 70, "so": 5964, "let": 2816, "stop": 983, "talkin": 276, "falsely": 3, "now": 3965, "hour": 119, "getting": 361, "late": 307, "all": 7435, "along": 200, "watchtower": 2, "princes": 2, "kept": 88, "view": 44, "while": 615, "women": 214, "came": 593, "went": 488, "barefoot": 7, "servants": 5, "outside": 138, "in": 13187, "cold": 388, "distance": 72, "wildcat": 3, "did": 903, "growl": 5, "two": 673, "riders": 11, "approaching": 5, "wind": 200, "began": 65, "howl": 8, "intro": 30, "joe": 45, "where": 1750, "you": 34759, "going": 743, "with": 6257, "gun": 231, "your": 9028, "hand": 625, "?": 4686, "verse": 261, "m": 7479, "down": 3457, "shoot": 154, "old": 743, "lady": 257, "know": 5643, "caught": 188, "her": 2870, "messing": 34, "around": 1258, "another": 766, "man": 2166, "ain": 1805, "cool": 246, "heard": 432, "shot": 330, "woman": 324, "ground": 218, "yes": 555, "town": 354, "gave": 299, "!": 3317, "guitar": 51, "solo": 30, "bridge": 131, "alright": 484, "one": 3349, "more": 1772, "time": 2526, "baby": 3837, "run": 529, "go": 3792, "well": 1059, "it": 16525, "south": 102, "mexico": 21, "free": 520, "find": 737, "me": 15874, "hangman": 1, "gonna": 1700, "put": 1136, "rope": 36, "better": 1105, "believe": 662, "right": 2077, "got": 5348, "outro": 31, "goodbye": 129, "everybody": 906, "ow": 67, "uh": 618, ...

### get_config 란 메소드를 통해서 토크나이저의 여러 정보를 불러왔다. <br/> 그 안에 word_counts 라는 딕셔너리가 보였다.

### 그래서 word_counts 딕셔너리만 불러와 보았다.

In [9]:
config_word = config['word_counts']

### 단어장에 쓰인 단어와 그 순서가 딕셔너리 구조로 저장되어있는 것을 확인했다.

In [10]:
### print(config_word) 너무 길어서 짤라서 올리겠습니다.

{"<'start'>": 156174, "there": 3040, "must": 509, "be": 6646, "some": 1847, "kind": 224, "of": 8964, "way": 2081, "outta": 145, "here": 1914, "<'end'>": 156174, "said": 1485, "the": 39314, "joker": 7, "to": 20368, "thief": 31, "s": 11126, "too": 1681, "much": 818, "confusion": 25, "i": 45489, "can": 5616, "t": 11207, "get": 4928, "no": 5083, "relief": 8, "business": 136, "men": 184, ",": 43415, "they": 5200, "drink": 264, "my": 15171, "wine": 106, "plowman": 1, "dig": 88, "earth": 180, "none": 102, "were": 912, "level": 45, "on": 9773, "mind": 871, "nobody": 588, "up": 6341, "at": 2758, "his": 1750, "word": 257, "hey": 1145, "reason": 210, "excited": 28, "he": 3802, "kindly": 12, "spoke": 47, "are": 2515, "many": 433, "among": 37, "us": 1353, "who": 1867, "feel": 1619, "that": 11539, "life": 1595, "is": 7287, "but": 5695, "a": 20617, "joke": 42, "and": 23103, "this": 5083, "not": 2793, "our": 1113, "fate": 70, "so": 5964, "let": 2816, "stop": 983, "talkin": 276, "falsely": 3, "now": 3965, "hour": 119, "getting": 361, "late": 307, "all": 7435, "along": 200, "watchtower": 2, "princes": 2, "kept": 88, "view": 44, "while": 615, "women": 214, "came": 593, "went": 488, "barefoot": 7, "servants": 5, "outside": 138, "in": 13187, "cold": 388, "distance": 72, "wildcat": 3, "did": 903, "growl": 5, "two": 673, "riders": 11, "approaching": 5, "wind": 200, "began": 65, "howl": 8, "intro": 30, "joe": 45, "where": 1750, "you": 34759, "going": 743, "with": 6257, "gun": 231, "your": 9028, "hand": 625, "?": 4686, "verse": 261, "m": 7479, "down": 3457, "shoot": 154, "old": 743, "lady": 257, "know": 5643, "caught": 188, "her": 2870, "messing": 34, "around": 1258, "another": 766, "man": 2166, "ain": 1805, "cool": 246, "heard": 432, "shot": 330, "woman": 324, "ground": 218, "yes": 555, "town": 354, "gave": 299, "!": 3317, "guitar": 51, "solo": 30, "bridge": 131, "alright": 484, "one": 3349, "more": 1772, "time": 2526, "baby": 3837, "run": 529, "go": 3792, "well": 1059, "it": 16525, "south": 102, "mexico": 21, "free": 520, "find": 737, "me": 15874, "hangman": 1, "gonna": 1700, "put": 1136, "rope": 36, "better": 1105, "believe": 662, "right": 2077, "got": 5348, "outro": 31, "goodbye": 129, "everybody": 906, "ow": 67, "uh": 618, "purple": 95, "haze": 18, "brain": 164, "lately": 47, "things": 911, "don": 5140, "seem": 178, "same": 515, "actin": 57, "funny": 122, "why": 1466, "excuse": 63, "kiss": 353, "sky": 532, ...

### 밸류값 순서로 정렬해 보았다.

In [34]:
sorted(config_word.items(), key=lambda x : x[1], reverse=True)

AttributeError: 'str' object has no attribute 'items'

### 에러가 발생했고, dict 타입이 아니라 str 타입이라는것 같다. 

### 혹시 몰라 타입을 조회해 보았다.

In [22]:
type(config_word)

str

### dict 구조의 str이라는것을 확인했다.

### 그래서 이러한 str을 dict으로 바꿔주는 함수가 있어 적용해 보았다.

In [23]:
config_word_d = eval(config_word)

In [24]:
type(config_word_d)

dict

### dict으로 잘 변경되었다.

In [27]:
sorted_word = sorted(config_word_d.items(), key=lambda x : x[1], reverse=True)
print(sorted_word[:10]) ### 너무 길어 10개만 올리겠습니다.

[('<start>', 156174), ('<end>', 156174), ('i', 45489), (',', 43415), ('the', 39314), ('you', 34759), ('and', 23103), ('a', 20617), ('to', 20368), ('it', 16525)]


### 정렬하고 출력해봤더니 단어장의 중요도와 순서는 단어의 개수로 판단하는것을 알수 있었다.

# 5. 소스 문장 생성

### 문장의 제일 뒤에 있는 단어와 앞에 있는 단어를 제거해 소스 문장을 생성한다.

In [8]:
# tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성합니다
# 마지막 토큰은 <end>가 아니라 <pad>일 가능성이 높습니다.
src_input = tensor[:, :-1]  
# tensor에서 <start>를 잘라내서 타겟 문장을 생성합니다.
tgt_input = tensor[:, 1:]    

print(src_input[0])
print(tgt_input[0])

[  2  62 271  27  94 546  20  86 743  90   3   0   0   0]
[ 62 271  27  94 546  20  86 743  90   3   0   0   0   0]


# 6. 데이터 분류

### 학습과 테스트를 위해 4:1 비율로 트레이닝셋과 테스트셋을 분리한다.

In [9]:
enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, tgt_input, test_size=0.2, random_state=5)

In [10]:
print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)

Source Train: (124939, 14)
Target Train: (124939, 14)


### 예제에 있는 단어보다 적게 나왔다.

### 이유는 정제된 문장이 단어가 0개인 경우가 있었는데 이 경우를 순서를 바꿔 정제 후 단어가 0개인 문장을 제거하니 더 적게 나왔다.

# 7. 버퍼사이즈와 배치사이즈를 정합니다.

### 검색을 하다 보니 배치사이즈와 에폭이 같이 나오는 경우가 많아 같이 검색해서 아래의 블로그를 참고했습니다.<br/>https://m.blog.naver.com/qbxlvnf11/221449297033

In [11]:
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

 # tokenizer가 구축한 단어사전 내 12000개와, 여기 포함되지 않은 0:<pad>를 포함하여 12001개
VOCAB_SIZE = tokenizer.num_words + 1   

# 준비한 데이터 소스로부터 데이터셋을 만듭니다
# 데이터셋에 대해서는 아래 문서를 참고하세요
# 자세히 알아둘수록 도움이 많이 되는 중요한 문서입니다
# https://www.tensorflow.org/api_docs/python/tf/data/Dataset
dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((256, 14), (256, 14)), types: (tf.int32, tf.int32)>

# 8. 모델 설계

### 저번 익스와 다르게 함수형인것을 TF 마스터 수업에서 조원의 도움으로 알게 되어 아래 블로그를 참고했습니다.<br/>https://www.tensorflow.org/guide/keras/functional?hl=ko

In [12]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 512
hidden_size = 2048
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [13]:
# 데이터셋에서 데이터 한 배치만 불러오는 방법입니다.
# 지금은 동작 원리에 너무 빠져들지 마세요~
for src_sample, tgt_sample in dataset.take(1): break

# 한 배치만 불러온 데이터를 모델에 넣어봅니다
model(src_sample)

<tf.Tensor: shape=(256, 14, 12001), dtype=float32, numpy=
array([[[ 1.24512342e-04,  5.83188485e-05, -1.61438948e-05, ...,
          1.38139090e-04,  3.00310512e-05, -1.84129050e-04],
        [ 5.46288735e-04,  1.03350227e-04,  3.64425272e-04, ...,
          2.15088381e-04, -4.69709230e-05, -2.62921996e-04],
        [ 1.19165110e-03,  5.69949625e-04,  3.74548894e-04, ...,
          2.98361963e-04,  2.57919019e-04, -8.58039930e-05],
        ...,
        [ 1.77767430e-03,  3.20719410e-04, -1.61111145e-03, ...,
          2.25838786e-03,  1.84631845e-05,  1.15446041e-04],
        [ 1.67242880e-03,  7.27213221e-04, -2.20229500e-03, ...,
          2.65908451e-03, -4.29614156e-05, -3.32463009e-04],
        [ 1.58718706e-03,  1.12879463e-03, -2.82804877e-03, ...,
          3.01699201e-03, -6.26632973e-05, -8.78968916e-04]],

       [[ 1.24512342e-04,  5.83188485e-05, -1.61438948e-05, ...,
          1.38139090e-04,  3.00310512e-05, -1.84129050e-04],
        [ 1.27307067e-04,  1.12269969e-04, -2

In [14]:
model.summary()

Model: "text_generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  6144512   
_________________________________________________________________
lstm (LSTM)                  multiple                  20979712  
_________________________________________________________________
lstm_1 (LSTM)                multiple                  33562624  
_________________________________________________________________
dense (Dense)                multiple                  24590049  
Total params: 85,276,897
Trainable params: 85,276,897
Non-trainable params: 0
_________________________________________________________________


In [15]:
# optimizer와 loss등은 차차 배웁니다
# 혹시 미리 알고 싶다면 아래 문서를 참고하세요
# https://www.tensorflow.org/api_docs/python/tf/keras/optimizers
# https://www.tensorflow.org/api_docs/python/tf/keras/losses
# 양이 상당히 많은 편이니 지금 보는 것은 추천하지 않습니다
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)
model.fit(enc_train, dec_train, epochs=4, validation_data=(enc_val, dec_val))

Epoch 1/4
3905/3905 [==============================] - 930s 238ms/step - loss: 3.2139 - val_loss: 2.6266
Epoch 2/4
3905/3905 [==============================] - 929s 238ms/step - loss: 2.4055 - val_loss: 2.3763
Epoch 3/4
3905/3905 [==============================] - 929s 238ms/step - loss: 1.9107 - val_loss: 2.2413
Epoch 4/4
3905/3905 [==============================] - 928s 238ms/step - loss: 1.5424 - val_loss: 2.1982


# epoch 5

embedding_size = 128
<br/>hidden_size = 1024
<br/>loss: 1.9839 - val_loss: 2.4296

embedding_size = 256
<br/>hidden_size = 1024
<br/>loss: 1.9083 - val_loss: 2.3947

embedding_size = 512
<br/>hidden_size = 1024
<br/>loss: 1.8408 - val_loss: 2.3680

embedding_size = 1024
<br/>hidden_size = 1024
<br/>loss: 1.8163 - val_loss: 2.3648

embedding_size = 256
<br/>hidden_size = 2048
<br/>loss: 1.3801 - val_loss: 2.2346

embedding_size = 512
<br/>hidden_size = 2048
<br/>loss: 1.3033 - val_loss: 2.2164

embedding_size = 1024
<br/>hidden_size = 2048
<br/>loss: loss: 1.2835 - val_loss: 2.2091

# epoch 4

embedding_size = 256
<br/>hidden_size = 2048
<br/>loss: 1.5385 - val_loss: 2.1943

### val_loss값을 2.2 이하로 낮추기 위해 처음엔 embedding_size와 hidden_size만 바꿔가며 학습하다 hidden_size가 커지면 오버피팅이 일어나서 epoch을 낮춰 2.2이하의 val_loss값을 구했습니다.

In [16]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 텐서로 변환합니다
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 단어 하나씩 예측해 문장을 만듭니다
    #    1. 입력받은 문장의 텐서를 입력합니다
    #    2. 예측된 값 중 가장 높은 확률인 word index를 뽑아냅니다
    #    3. 2에서 예측된 word index를 문장 뒤에 붙입니다
    #    4. 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성을 마칩니다
    while True:
        # 1
        predict = model(test_tensor) 
        # 2
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        # 3 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 4
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환합니다 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated

# 9. 문장을 만들어 보았습니다.

In [17]:
generate_text(model, tokenizer, init_sentence="<start> i love", max_len=20)

'<start> i love ma little nasty girl <end> '

### 완전한 문장은 아니지만 어느정도 의미는 알것 같습니다.

# 회고

### 임베딩 사이즈와 히든 사이즈를 바꿔가며 학습속도가 엄청나게 달라지는것을 구글링과 직접 몸으로 느낄수 있었습니다.

### 하이퍼 파라미터의 중요도를 알수 있었다고 생각하고, 학습속도가 느리더라도 테스트속도와 loss값만 낮다면 좋다고 생각하지만 테스트 속도만 낮추는 방법은 좀더 고민해 봐야 할것 같습니다.